# Chapter 10. Design Patterns with First-Class Functions

Mặc dù các mẫu thiết kế độc lập với ngôn ngữ nhưng điều đó không có nghĩa là mọi mẫu đều áp dụng cho mọi ngôn ngữ.
- 16/23 mẫu trong cuốn sách Mẫu thiết kế ban đầu trở nên "vô hình hoặc đơn giản hơn" trong ngôn ngữ động

Mục tiêu của chương này là chỉ ra cách—trong một số trường hợp—các hàm có thể thực hiện công việc giống như các lớp, với mã dễ đọc và ngắn gọn hơn.


## Case Study: Refactoring Strategy

functions as first-class objects

Implement Strategy using the “classic” structure described in Design Patterns.

### Classic Strategy

Hình 10-1. Sơ đồ lớp UML để xử lý  order discount được triển khai với Strategy design pattern.

![img.png](image/10-1.png)

- Khách có > 1000đ , giảm 5%/order
- Mua >20 items/order, giảm 10%
- Mua >10 items riêng biệt, giảm 7%

Strategy pattern:

"Định nghĩa một nhóm các thuật toán, đóng gói từng thuật toán và làm cho chúng có thể hoán đổi cho nhau. Strategy cho phép thuật toán thay đổi độc lập với các client sử dụng nó."

In [1]:
# Mặc định trong Python sẽ không cung cấp Abstract class cho chúng ta sử dụng.
# Nhưng Python có một mô-đun gọi là Abstract Base Classes (ABC) để giúp chúng ta làm điều đó.
from abc import ABC, abstractmethod
from collections.abc import Sequence
from decimal import Decimal
from typing import NamedTuple, Optional


class Customer(NamedTuple):
    name: str
    fidelity: int # điểm


class LineItem(NamedTuple):
    product: str
    quantity: int
    price: Decimal

    def total(self) -> Decimal:
        return self.price * self.quantity


class Order(NamedTuple):  # the Context
    customer: Customer
    cart: Sequence[LineItem]
    promotion: Optional['Promotion'] = None

    def total(self) -> Decimal:
        totals = (item.total() for item in self.cart)
        return sum(totals, start=Decimal(0))

    def due(self) -> Decimal: # Giá khi đã giảm
        if self.promotion is None:
            discount = Decimal(0)
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount

    def __repr__(self):
        return f'<Order total: {self.total():.2f} due: {self.due():.2f}>'


class Promotion(ABC):  # the Strategy: an abstract base class
    @abstractmethod
    def discount(self, order: Order) -> Decimal:
        """Return discount as a positive dollar amount"""



class FidelityPromo(Promotion):  # first Concrete Strategy
    """5% discount for customers with 1000 or more fidelity points"""

    def discount(self, order: Order) -> Decimal:
        rate = Decimal('0.05')
        if order.customer.fidelity >= 1000:
            return order.total() * rate
        return Decimal(0)


class BulkItemPromo(Promotion):  # second Concrete Strategy
    """10% discount for each LineItem with 20 or more units"""

    def discount(self, order: Order) -> Decimal:
        discount = Decimal(0)
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * Decimal('0.1')
        return discount


class LargeOrderPromo(Promotion):  # third Concrete Strategy
    """7% discount for orders with 10 or more distinct items"""

    def discount(self, order: Order) -> Decimal:
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * Decimal('0.07')
        return Decimal(0)

In [2]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)

cart = (LineItem('banana', 4, Decimal('.5')),
        LineItem('apple', 10, Decimal('1.5')),
        LineItem('watermelon', 5, Decimal(5)))

Order(joe, cart, FidelityPromo())

<Order total: 42.00 due: 42.00>

In [3]:
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

In [4]:
banana_cart = (LineItem('banana', 30, Decimal('.5')),
               LineItem('apple', 10, Decimal('1.5')))

Order(joe, banana_cart, BulkItemPromo())

<Order total: 30.00 due: 28.50>

In [5]:
long_cart = tuple(LineItem(str(sku), 1, Decimal(1)) for sku in range(10))

long_cart

(LineItem(product='0', quantity=1, price=Decimal('1')),
 LineItem(product='1', quantity=1, price=Decimal('1')),
 LineItem(product='2', quantity=1, price=Decimal('1')),
 LineItem(product='3', quantity=1, price=Decimal('1')),
 LineItem(product='4', quantity=1, price=Decimal('1')),
 LineItem(product='5', quantity=1, price=Decimal('1')),
 LineItem(product='6', quantity=1, price=Decimal('1')),
 LineItem(product='7', quantity=1, price=Decimal('1')),
 LineItem(product='8', quantity=1, price=Decimal('1')),
 LineItem(product='9', quantity=1, price=Decimal('1')))

In [6]:
Order(joe, long_cart, LargeOrderPromo())

<Order total: 10.00 due: 9.30>

Ví dụ 10-1 hoạt động hoàn hảo, nhưng chức năng tương tự có thể được triển khai với ít mã hơn trong Python bằng cách sử dụng functions as objects.

### Function-Oriented Strategy

Each concrete strategy in Example 10-1 is a class with a single method + no state --> a lot like plain functions

In [7]:
# Example 10-3. Order class with discount strategies implemented as functions

from collections.abc import Sequence
from dataclasses import dataclass
from decimal import Decimal
from typing import Optional, Callable, NamedTuple


class Customer(NamedTuple):
    name: str
    fidelity: int


class LineItem(NamedTuple):
    product: str
    quantity: int
    price: Decimal

    def total(self):
        return self.price * self.quantity


@dataclass(frozen=True)
class Order:  # the Context
    customer: Customer
    cart: Sequence[LineItem]
    # This type hint says:
    # promotion may be None, or it may be a callable that takes an Order argument and returns a Decimal.
    promotion: Optional[Callable[['Order'], Decimal]] = None

    def total(self) -> Decimal:
        totals = (item.total() for item in self.cart)
        return sum(totals, start=Decimal(0))

    def due(self) -> Decimal:
        if self.promotion is None:
            discount = Decimal(0)
        else:
            discount = self.promotion(self)
        return self.total() - discount

    def __repr__(self):
        return f'<Order total: {self.total():.2f} due: {self.due():.2f}>'

# No abstract class

def fidelity_promo(order: Order) -> Decimal:
    """5% discount for customers with 1000 or more fidelity points"""
    if order.customer.fidelity >= 1000:
        return order.total() * Decimal('0.05')
    return Decimal(0)

def bulk_item_promo(order: Order) -> Decimal:
    """10% discount for each LineItem with 20 or more units"""
    discount = Decimal(0)
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * Decimal('0.1')
    return discount


def large_order_promo(order: Order) -> Decimal:
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * Decimal('0.07')
    return Decimal(0)

WHY SELF.PROMOTION(SELF)?

In the `Order` class, `promotion` is not a method. It’s an instance attribute that happens to be callable.
- `self.promotion`, truy xuất giá trị có thể gọi đó
- Để gọi nó, chúng ta phải cung cấp một thể hiện của `Order`, trong trường hợp này là `self`.

In [8]:
Order(joe, cart, fidelity_promo)

<Order total: 42.00 due: 42.00>

In [9]:
Order(ann, cart, fidelity_promo)

<Order total: 42.00 due: 39.90>

In [10]:
Order(joe, long_cart, large_order_promo)

<Order total: 10.00 due: 9.30>

--> “Strategy objects often make good flyweights.”
(Flyweight là một đối tượng được chia sẻ có thể được sử dụng đồng thời trong nhiều ngữ cảnh.)

VD: không cần tạo concrete strategy object mới với mỗi đơn đặt hàng mới

--> "Một nhược điểm của Strategy pattern—runtime cost"

Sẽ cần kết hợp Strategy và Flyweight design patterns với concrete strategies phức tạp (lưu state).

Nhưng thông thường, concrete strategies ko có state, nó thường xử lý data từ context hơn.
- dùng functions thay vì class (như ví dụ 10-3)
- không cần Flyweight

Okay. Giờ thêm chức năng mới "metastrategy", chọn mức giảm giá tốt nhất cho 1 `Order`:
- Tạo `best_promo` function: `Order(joe, long_cart, best_promo)`

### Choosing the Best Strategy: Simple Approach

In [11]:
# Example 10-6. best_promo finds the maximum discount iterating over a list of functions

promos = [fidelity_promo, bulk_item_promo, large_order_promo]


def best_promo(order: Order) -> Decimal:
    """Compute the best discount available"""
    # Using a generator expression,
    # we apply each of the functions from promos to the order,
    # and return the maximum discount computed.
    return max(promo(order) for promo in promos)

Order(joe, long_cart, best_promo)

<Order total: 10.00 due: 9.30>

Vấn đề: Mỗi lần có 1 promotion strategy mới, lại phải thêm vào promos list

### Finding Strategies in a Module

Modules in Python are also first-class objects.

In [16]:
# Example 10-7. The promos list is built by introspection (sự quan sát) of the module global namespace

from decimal import Decimal
from strategy import Order

# Import the promotion functions
# so they are available in the global namespace
from strategy import (
    fidelity_promo, bulk_item_promo, large_order_promo
)

# filter out best_promo itself,
# to avoid an infinite recursion when best_promo is called.
promos = [promo for name, promo in globals().items()
                if name.endswith('_promo') and
                name != 'best_promo'
]


def best_promo1(order: Order) -> Decimal:
    """Compute the best discount available"""
    return max(promo(order) for promo in promos)

Order(joe, long_cart, best_promo)

<Order total: 10.00 due: 9.30>

In [17]:
# Example 10-8. The promos list is built by introspection of a new promotions module

from decimal import Decimal
import inspect

from strategy import Order
import promotions


promos = [func for _, func in inspect.getmembers(promotions, inspect.isfunction)]


def best_promo(order: Order) -> Decimal:
    """Compute the best discount available"""
    return max(promo(order) for promo in promos)

Order(joe, long_cart, best_promo)

<Order total: 10.00 due: 9.30>

Phần sau chỉ cách để filter func rõ ràng và chính xác hơn bằng cách sử dụng `deco`

## Decorator-Enhanced Strategy Pattern

Vấn đề là ở Ví dụ 10.6 , khi thêm 1 promotional strategy mới, quên add và list promos
- Ví dụ 10-9 giải quyết vấn đề này bằng kỹ thuật có trong "Registration Decorators".


In [18]:
# Example 10-9. The promos list is filled by the Promotion decorator

Promotion = Callable[[Order], Decimal]

promos: list[Promotion] = []

def promotion(promo: Promotion) -> Promotion:
    promos.append(promo)
    return promo


def best_promo(order: Order) -> Decimal:
    """Compute the best discount available"""
    return max(promo(order) for promo in promos)

@promotion
def fidelity(order: Order) -> Decimal:
    """5% discount for customers with 1000 or more fidelity points"""
    if order.customer.fidelity >= 1000:
        return order.total() * Decimal('0.05')
    return Decimal(0)


@promotion
def bulk_item(order: Order) -> Decimal:
    """10% discount for each LineItem with 20 or more units"""
    discount = Decimal(0)
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * Decimal('0.1')
    return discount


@promotion
def large_order(order: Order) -> Decimal:
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * Decimal('0.07')
    return Decimal(0)

promos

[<function __main__.fidelity(order: strategy.Order) -> decimal.Decimal>,
 <function __main__.bulk_item(order: strategy.Order) -> decimal.Decimal>,
 <function __main__.large_order(order: strategy.Order) -> decimal.Decimal>]

## The Command Pattern

Command, một mẫu design pattern khác, mà có thể được đơn giản hóa bằng cách sử dụng các hàm được truyền dưới dạng đối số

Hình 10-2: Sắp xếp các class trong Command pattern
- Cho menu-driven text editor
- Mỗi command có thể có receiver khác nhau:
    - `PasteCommand`, receiver là Document
    - `OpenCommand`, receiver là Application


![img.png](image/10-2.png)

Mục tiêu của Command
- Tách invoker (1 object gọi 1 operation) với receiver (object thực hiện)
    - Ở ví dụ:
        - Mỗi invoker là là một mục menu trong ứng dụng đồ họa
        - Mỗi receiver là Docmument/Application
- Ideal: Put Command object ở giữa
    - Implement 1 interface với 1 method duy nhất
        - invoker ko cần biết interface của receiver
        - receiver có thể được dùng qua các Command subclasses khác nhau
- “Commands are an object-oriented replacement for callbacks.” (not always)

Như Strategy pattern ở phần trước:
- Thay vì tạo 1 Command instance -> dùng 1 function
- Thay vì gọi `command.execute()` -> dùng `command()` via `__call__` method.

In [20]:
# Example 10-10. Each instance of MacroCommand has an internal list of commands

class MacroCommand:
    """A command that executes a list of commands"""

    # m = MacroCommand()
    def __init__(self, commands):
        self.commands = list(commands)

    # m()
    def __call__(self):
        for command in self.commands:
            command()

More advanced uses of the Command pattern—to support undo

After all, every Python callable implements a single-method interface, and that method is named `__call__`.

# Chapter Summary

- 16 trong số 23 mẫu có cách triển khai trong Lisp hoặc Dylan (Dynamic Languages) đơn giản hơn về mặt chất lượng so với trong C++. (trang 9 https://fpy.li/10-4)
- Sử dụng Strategy pattern là 1 starting point
    - Giải pháp thêm vào là first-class functions.
- Funcs và callable objects cung cấp một cách tự nhiên hơn để implement callbacks (trong Python) hơn là dùng Strategy/Command patterns.
    - Tái cấu trúc Strategy / thảo luận Command ở trên là 1 ví dụ tổng quát khi cần:
        - Có yêu cầu viết design pattern/API mà implement 1 interface với single method và method đó rất chung chung (VD: “execute,” “run,” “do_it.”)
                - Tốn ít code hơn với việc sử dụng  functions as first-class objects.